In [1]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [2]:
# import os
# os.getcwd()
# #path = os.join(os.getcwd(), '/drive/app')
#
# path = r'/content/drive/My Drive/app'
# os.chdir(path)
# print(os.getcwd())
# print(os.listdir())
# print(os.path.isdir('logs'))

In [3]:
import funciones_wgan as f
import numpy as np
import cv2
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers.convolutional import Convolution2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.datasets import mnist
from keras import backend as K
import matplotlib.pyplot as plt
%matplotlib inline
from time import time
import tensorflow as tf
import os
from keras.callbacks import TensorBoard
from datetime import datetime

Using TensorFlow backend.


In [4]:
print(os.getcwd())

/home/jupyter/UNEDWMH


# Funciones útiles

In [5]:
def write_list(lista, FILENAME, mode='a+'):
    with open(FILENAME, mode) as f:
        for item in lista:
            f.write("%s\n" % item)

# Variables generales

In [6]:
EPOCHS = 10000
BATCH_SIZE = 8
# The training ratio is the number of discriminator updates
# per generator update. The paper uses 5.
TRAINING_RATIO = 8
GRADIENT_PENALTY_WEIGHT = 10  # As per the paper
INPUT_LEN = 128
output_dir = 'output/'
#discriminator_weights = 'Weights/discriminator_epoch_1750.h5'
#generator_weights = 'Weights/generator_epoch_1750.h5'
muestra = None # Si queremos coger una muestra de las imágenes. None para no utilizar
intervalo_guardado = 50
file_discriminator = output_dir + 'LOSS/disc_loss.txt'
file_generator = output_dir + 'LOSS/gen_loss.txt'
log_path = 'logs/'

imagenes_muestra = None  # None para leer el completo

kernel_size_generator = 4

In [7]:
try:
    initial_epoch = int(generator_weights.split('_')[2].split('.')[0]) + 1
except:
    initial_epoch = 0

final_epoch = initial_epoch + EPOCHS - 1

# Discriminador

In [8]:
def make_discriminator():
    """Creates a discriminator model that takes an image as input and outputs a single
    value, representing whether the input is real or generated. Unlike normal GANs, the
    output is not sigmoid and does not represent a probability! Instead, the output
    should be as large and negative as possible for generated inputs and as large and
    positive as possible for real inputs.
    Note that the improved WGAN paper suggests that BatchNormalization should not be
    used in the discriminator."""
    
    model = Sequential()
    model.add(Convolution2D(32, 5, padding='same', strides=[2, 2], input_shape=(256, 256, 3)))
    model.add(LeakyReLU())

    model.add(Convolution2D(64, 5, kernel_initializer='he_normal', strides=[2, 2], padding='same'))
    model.add(LeakyReLU())

    model.add(Convolution2D(128, 5, kernel_initializer='he_normal', padding='same', strides=[2, 2]))
    model.add(LeakyReLU())

    model.add(Convolution2D(256, 5, kernel_initializer='he_normal', padding='same', strides=[2, 2]))
    model.add(LeakyReLU())

    model.add(Convolution2D(512, 5, kernel_initializer='he_normal', padding='same', strides=[2, 2]))
    model.add(LeakyReLU())

    model.add(Convolution2D(1024, 5, kernel_initializer='he_normal', padding='same', strides=[2, 2]))
    model.add(LeakyReLU())

    model.add(Flatten())
    #model.add(Dense(1024 * 4 * 4, kernel_initializer='he_normal'))
    #model.add(LeakyReLU())
    model.add(Dense(1, kernel_initializer='he_normal'))
    
    return model

# Generador

In [9]:
def make_generator():
    """Creates a generator model that takes a 128-dimensional noise vector as a "seed",
    and outputs images of size 256x256x3."""
    model = Sequential()
    
    model.add(Dense(4 * 4 * 2048, input_dim=INPUT_LEN))
    #model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Reshape((4, 4, 2048), input_shape=(4 * 4 * 2048,)))
    bn_axis = -1

    model.add(Conv2DTranspose(1024, kernel_size_generator, strides=2, padding='same'))
    #model.add(BatchNormalization(axis=bn_axis))
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(512, kernel_size_generator, strides=2, padding='same'))
    #model.add(BatchNormalization(axis=bn_axis))
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(256, kernel_size_generator, strides=2, padding='same'))
    #model.add(BatchNormalization(axis=bn_axis))
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(128, kernel_size_generator, strides=2, padding='same'))
    #model.add(BatchNormalization(axis=bn_axis))
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(64, kernel_size_generator, strides=2, padding='same'))
    #model.add(BatchNormalization(axis=bn_axis))
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(3, kernel_size_generator, strides=2, padding='same', activation='tanh'))
    # El output de esta última es 256x256x3
    
    return model

# Carga de datos

El archivo total_three_datasets_sorted.npy es una matriz que ya tiene las tres capas:
* 0: T1
* 1: FLAIR
* 2: Máscara

In [10]:
try:
    #images = np.load('total_three_datasets_sorted.npy')
    images = np.load('muestra_seleccionada.npy')
except:
    images = np.load('images_three_datasets_sorted.npy')
    masks = np.load('masks_three_datasets_sorted.npy')
    
    # Normalizado entre -1 y +1. Esto lo hace sobre toda la imagen, no sobre la capa T1 o FLAIR por separado.
    # No tengo muy claro que sea correcto
    images = [2.*(image - np.min(image))/np.ptp(image) - 1 for image in images]
        
    images = np.concatenate((images, masks), axis=3)
    
    # El generador toma imágenes 256x256x3. Como las tenemos 200x200, hay que redimensionarlas:
    dim_final = (256, 256)
    images = np.array([cv2.resize(image, dim_final, interpolation = cv2.INTER_AREA) for image in images])
    
    np.save('total_three_datasets_sorted.npy', images)
    del masks

In [11]:
muestra

Si se trabaja con Tensorflow está bien porque los canales están en la última dimensión del array.

In [12]:
if imagenes_muestra is not None:
    images = np.load(imagenes_muestra)
    #random = np.random.uniform(0, 0.1, images[...,2].shape).astype('float32')
    random = np.random.normal(0.5, 0.1, images[...,2].shape).astype('float32')
    images[...,2] = images[...,2].astype(float)
    np.putmask(images[...,2], images[...,2] == 1, random)
    np.putmask(images[...,2], images[...,2] == 0, -random)
    print('Imágenes cargadas del archivo ' + imagenes_muestra)
else:
    if muestra is not None:
        np.random.seed(1234)  
        images_shuff = images[:]
        np.random.shuffle(images_shuff)
        images_shuff = images_shuff[0:muestra,...]
        images = images_shuff[:]
        print(type(images[0,0,0,0]))
        #random = np.random.uniform(0, 0.1, images[...,2].shape).astype('float32')
        random = np.random.normal(0.5, 0.1, images[...,2].shape).astype('float32')
        images[...,2] = images[...,2].astype(float)
        np.putmask(images[...,2], images[...,2] == 1, random)
        np.putmask(images[...,2], images[...,2] == 0, -random)        
        np.save('muestra_' + str(muestra) + '_' + datetime.now().strftime("%Y%m%d%H%M%S") + '.npy', images)
        


In [13]:
print(np.amin(images[34,...,0]), np.amin(images[34,...,1]))

-0.9252150058746338 -1.0


In [14]:
n_images = images.shape[0]
print(n_images)

180


# Redes

Inicialización de generador y discriminador.

In [15]:
generator = make_generator()
discriminator = make_discriminator()

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
try:
    generator.load_weights(generator_weights)
    discriminator.load_weights(discriminator_weights)
except:
    print('Ha habido un error al cargar los pesos')

Ha habido un error al cargar los pesos


### Generador

In [17]:
# The generator_model is used when we want to train the generator layers.
# As such, we ensure that the discriminator layers are not trainable.
# Note that once we compile this model, updating .trainable will have no effect within
# it. As such, it won't cause problems if we later set discriminator.trainable = True
# for the discriminator_model, as long as we compile the generator_model first.
for layer in discriminator.layers:
    layer.trainable = False
    
discriminator.trainable = False
generator_input = Input(shape=(INPUT_LEN,))
generator_layers = generator(generator_input)
discriminator_layers_for_generator = discriminator(generator_layers)
generator_model = Model(inputs=[generator_input], outputs=[discriminator_layers_for_generator])
# We use the Adam paramaters from Gulrajani et al.
generator_model.compile(optimizer=Adam(0.0001, beta_1=0.5, beta_2=0.9), loss=f.wasserstein_loss)

### Discriminador

In [18]:
# Now that the generator_model is compiled, we can make the discriminator
# layers trainable.
for layer in discriminator.layers:
    layer.trainable = True
    
for layer in generator.layers:
    layer.trainable = False
    
discriminator.trainable = True
generator.trainable = False

# The discriminator_model is more complex. It takes both real image samples and random
# noise seeds as input. The noise seed is run through the generator model to get
# generated images. Both real and generated images are then run through the
# discriminator. Although we could concatenate the real and generated images into a
# single tensor, we don't (see model compilation for why).
real_samples = Input(shape=images.shape[1:])
generator_input_for_discriminator = Input(shape=(INPUT_LEN,))
generated_samples_for_discriminator = generator(generator_input_for_discriminator)
discriminator_output_from_generator = discriminator(generated_samples_for_discriminator)
discriminator_output_from_real_samples = discriminator(real_samples)

In [ ]:
# We also need to generate weighted-averages of real and generated samples,
# to use for the gradient norm penalty.
averaged_samples = f.RandomWeightedAverage()([real_samples,
                                            generated_samples_for_discriminator])
# We then run these samples through the discriminator as well. Note that we never
# really use the discriminator output for these samples - we're only running them to
# get the gradient norm for the gradient penalty loss.
averaged_samples_out = discriminator(averaged_samples)

In [ ]:
# The gradient penalty loss function requires the input averaged samples to get
# gradients. However, Keras loss functions can only have two arguments, y_true and
# y_pred. We get around this by making a partial() of the function with the averaged
# samples here.
partial_gp_loss = f.partial(f.gradient_penalty_loss,
                            averaged_samples=averaged_samples, gradient_penalty_weight=GRADIENT_PENALTY_WEIGHT)
# Functions need names or Keras will throw an error
partial_gp_loss.__name__ = 'gradient_penalty'

In [ ]:
# Keras requires that inputs and outputs have the same number of samples. This is why
# we didn't concatenate the real samples and generated samples before passing them to
# the discriminator: If we had, it would create an output with 2 * BATCH_SIZE samples,
# while the output of the "averaged" samples for gradient penalty
# would have only BATCH_SIZE samples.

# If we don't concatenate the real and generated samples, however, we get three
# outputs: One of the generated samples, one of the real samples, and one of the
# averaged samples, all of size BATCH_SIZE. This works neatly!
discriminator_model = Model(inputs=[real_samples, generator_input_for_discriminator],
                            outputs=[discriminator_output_from_real_samples, discriminator_output_from_generator, averaged_samples_out])

In [ ]:
# We use the Adam paramaters from Gulrajani et al. We use the Wasserstein loss for both
# the real and generated samples, and the gradient penalty loss for the averaged samples
discriminator_model.compile(optimizer=Adam(0.0001, beta_1=0.5, beta_2=0.9),
                            loss=[f.wasserstein_loss, f.wasserstein_loss, partial_gp_loss])

# We make three label vectors for training. positive_y is the label vector for real
# samples, with value 1. negative_y is the label vector for generated samples, with
# value -1. The dummy_y vector is passed to the gradient_penalty loss function and
# is not used.
positive_y = np.ones((BATCH_SIZE, 1), dtype=np.float32)
negative_y = -positive_y
dummy_y = np.zeros((BATCH_SIZE, 1), dtype=np.float32)

In [ ]:
n_minibatches = int(n_images // (BATCH_SIZE * TRAINING_RATIO))
minibatch_epochs = initial_epoch * n_minibatches
training_ratio_epoch = initial_epoch * n_minibatches * TRAINING_RATIO

print("Number of batches: ", int(n_images // BATCH_SIZE))
print('Tenemos ', int(n_images // (BATCH_SIZE * TRAINING_RATIO)), ' minibatches.')

callback = TensorBoard(log_path)
callback.set_model(generator_model)
#file_writer = tf.summary.FileWriter(log_path)

discriminator_loss_mean = []
generator_loss_mean = []

for epoch in range(initial_epoch, final_epoch):
    start = time()
    np.random.shuffle(images)
    print("Epoch: ", epoch)
    
    discriminator_loss_epoch = []
    generator_loss_epoch = []
    
    minibatches_size = BATCH_SIZE * TRAINING_RATIO
    
    for i in range(int(n_images // (BATCH_SIZE * TRAINING_RATIO))):
        discriminator_minibatches = images[i * minibatches_size: (i + 1) * minibatches_size]
        
        for j in range(TRAINING_RATIO):
            image_batch = discriminator_minibatches[j * BATCH_SIZE: (j + 1) * BATCH_SIZE]
            noise = np.random.normal(0, 1, (BATCH_SIZE, INPUT_LEN)).astype(np.float32)
            #noise = np.random.uniform(-1,1,(BATCH_SIZE, INPUT_LEN)).astype(np.float32)
            discriminator_loss_val = discriminator_model.train_on_batch([image_batch, noise], [positive_y, negative_y, dummy_y])
            discriminator_loss_epoch.append(discriminator_loss_val)

            # PARA TENSORBOARD
            f.write_log(callback, ['d_loss', 'd_loss_real', 'd_loss_fake'], [
                discriminator_loss_val[0],
                discriminator_loss_val[1],
                discriminator_loss_val[2]
            ], training_ratio_epoch)

            training_ratio_epoch += 1
        
        #generator_loss_val = generator_model.train_on_batch(np.random.uniform(-1,1,(BATCH_SIZE, INPUT_LEN)), positive_y)
        generator_loss_val = generator_model.train_on_batch(np.random.normal(0, 1, (BATCH_SIZE, INPUT_LEN)), positive_y)
        generator_loss_epoch.append(generator_loss_val)

        # ESCRIBIR PARA TENSORBOARD
        f.write_log(callback, ['g_loss'], [generator_loss_val], minibatch_epochs)

        minibatch_epochs += 1
        
    generator_loss_mean.append(np.mean(generator_loss_epoch))
    discriminator_loss_mean.append(np.mean(discriminator_loss_epoch, axis=0))
    
    print('Epoch ' + str(epoch) + ' took ' + str(time() - start))
       
    if epoch % intervalo_guardado == 0:
        print('Saving weights')
        generator.save_weights('Weights/generator_epoch_' + str(epoch) + '.h5')
        discriminator.save_weights('Weights/discriminator_epoch_' + str(epoch) + '.h5')
        print('Weights saved')
        
        f.sample_best_images(generator, discriminator, output_dir, epoch, 10)
        image_board = f.sample_best_images(generator, discriminator, output_dir, epoch, 1, save = False)
        #with file_writer.as_default():
        #    tf.summary.image('Epoch_' + str(epoch), image_board, step=0
        #tbc.save_image('Epoch_' + str(epoch), image_board)

        try:
            os.remove('./Weights/discriminator_epoch_' + str(epoch-intervalo_guardado) + '.h5')
        except:
            pass
        try:
            os.remove('./Weights/generator_epoch_' + str(epoch-intervalo_guardado) + '.h5')
        except:
            pass
          
        write_list(discriminator_loss_mean, file_discriminator)
        write_list(generator_loss_mean, file_generator)
        
        discriminator_loss_mean = []
        generator_loss_mean = []

Number of batches:  22
Tenemos  2  minibatches.
Epoch:  0
Instructions for updating:
Use tf.cast instead.
Epoch 0 took 9.25908899307251
Saving weights
Weights saved
(100, 256, 256, 3)
output/T1/epoch_0.png
output/Mask/epoch_0.png
(100, 256, 256, 3)
Epoch:  1
Epoch 1 took 2.1369307041168213
Epoch:  2
Epoch 2 took 2.1353938579559326
Epoch:  3
Epoch 3 took 2.138178586959839
Epoch:  4
Epoch 4 took 2.1390702724456787
Epoch:  5
Epoch 5 took 2.136953592300415
Epoch:  6
Epoch 6 took 2.1391940116882324
Epoch:  7
Epoch 7 took 2.1401443481445312
Epoch:  8
Epoch 8 took 2.137422561645508
Epoch:  9
Epoch 9 took 2.134977102279663
Epoch:  10
Epoch 10 took 2.134141445159912
Epoch:  11
Epoch 11 took 2.132298231124878
Epoch:  12
Epoch 12 took 2.140089988708496
Epoch:  13
Epoch 13 took 2.161940097808838
Epoch:  14
Epoch 14 took 2.1534953117370605
Epoch:  15
Epoch 15 took 2.1583714485168457
Epoch:  16
Epoch 16 took 2.1561670303344727
Epoch:  17
Epoch 17 took 2.1545374393463135
Epoch:  18
Epoch 18 took 2.14

In [ ]:
image_board_FLAIR, image_board_T1 = f.sample_best_images(generator, discriminator, output_dir, epoch, 1, save = False)

In [ ]:
image_board_flair